In [1]:
!pip install optuna

     |████████████████████████████████| 308 kB 15.9 MB/s 
     |████████████████████████████████| 210 kB 41.6 MB/s 
     |████████████████████████████████| 81 kB 3.1 MB/s 
     |████████████████████████████████| 78 kB 5.9 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
     |████████████████████████████████| 150 kB 43.2 MB/s 
     |████████████████████████████████| 113 kB 47.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=b0081baab9aab609242d278a820427ddc1744bf6e04f6fc302ec6c8d346e70e8
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import warnings
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline 

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.utils import resample

import optuna

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
train=pd.read_csv(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/trainning/funix_train.csv")
val=pd.read_csv(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/trainning/funix_val.csv")
test=pd.read_csv(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/trainning/funix_test.csv")

In [5]:
def reduce_memory_usage(df):   
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                    
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [6]:

def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [7]:
train = reduce_memory_usage(train)
val = reduce_memory_usage(val)
test = reduce_memory_usage(test)

Memory usage of dataframe is 1.166229248046875 MB
Memory usage of dataframe after reduction 0.3381690979003906 MB
Reduced by 71.00320554755986 % 
Memory usage of dataframe is 0.2066650390625 MB
Memory usage of dataframe after reduction 0.058624267578125 MB
Reduced by 71.63319551092735 % 
Memory usage of dataframe is 0.242523193359375 MB
Memory usage of dataframe after reduction 0.06910324096679688 MB
Reduced by 71.50654334969171 % 


In [8]:
train.head()

,ID,MER_TYPE,BRANCH,CURR_NUM,DAY_REINSTATED,DAYS_AVGE,FL_LIM1,FL_LIM3,FL_LIM4,GROSS_SALE,...,DAY_CLOSE_YEAR,DAY_CLOSE_MONTH,DAY_CLOSE_DAY,DAY_START_YEAR,DAY_START_MONTH,DAY_START_DAY,DAY_SUBM_YEAR,DAY_SUBM_MONTH,DAY_SUBM_DAY,MERCH_FR
0,2208,5411,126,704,0,1,150,0,5000,671981120.0,...,2099,12,31,1970,1,1,1970,1,1,1
1,53,7011,753,704,0,2,150,0,0,0.0,...,2016,11,14,1970,1,1,1970,1,1,0
2,52,5411,128,704,0,2,150,0,5000,204670160.0,...,2099,12,31,1970,1,1,1970,1,1,0
3,163,5977,741,704,0,3,150,0,0,0.0,...,2016,10,3,1970,1,1,1970,1,1,0
4,657,5411,721,704,0,1,150,0,5000,0.0,...,2020,7,24,1970,1,1,1970,1,1,0


In [9]:
useful_features = [c for c in train.columns if ((c not in ['ID', 'MERCH_FR', 'BRANCH']) and ("DAY_" not in c))] 
# list_cat_org = get_list_cat_columns() 
# object_cols = [c for c in useful_features if c in list_cat_org]
object_cols = ['CURR_NUM', 'MP_IND', 'PAY_METHOD', 'HAS_TXN_LESS_15S']
num_cols = [c for c in useful_features if c not in object_cols]
# df_test = val[useful_features]

In [10]:
x_train = train[useful_features]
x_val = val[useful_features]
x_test = test[useful_features]
y_train = train["MERCH_FR"]
y_val = val["MERCH_FR"]
y_test = test["MERCH_FR"]

In [11]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler, OneHotEncoder
from imblearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler() # ('scaler', MinMaxScaler()    
    )])

# cat_label_cols = ['CST_PERF_ST', 'CST_MKT_SEG']
cat_ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_val')),
    ('encoder', OrdinalEncoder() 
    )])

# cat_onehot_cols = [f for f in cat_cols if f not in cat_label_cols]
cat_onehot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_val')),
    ('encoder', OneHotEncoder(handle_unknown = 'ignore', sparse=False) 
    )])

# over sampling 
oversample = SMOTE() # ADASYN()
preprocessor = ColumnTransformer(
    transformers=[
                  ('num', numeric_transformer, num_cols), 
                  #('cat_label', cat_ordinal_transformer, cat_label_cols)
                  #('cat_onehot', cat_onehot_transformer, ['CLOSE_RSN','CHIP_YN']) # ['CLOSE_RSN', 'COMM_FREQ']
                 ], remainder='drop')

In [12]:
# preprocessor.fit(train[useful_features])
preprocessor.fit(x_train)

x_train_tf = pd.DataFrame(preprocessor.transform(x_train), columns = get_feature_names(preprocessor)) 
x_val_tf = pd.DataFrame(preprocessor.transform(x_val), columns = get_feature_names(preprocessor)) 
x_test_tf = pd.DataFrame(preprocessor.transform(x_test), columns = get_feature_names(preprocessor)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available


In [13]:
def run(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)    
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    max_features = trial.suggest_int('max_leaf_nodes', 2, 30)    
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50) 
    min_samples_leaf = trial.suggest_int('min_samples_split', 2, 50)

    model = RandomForestClassifier(
        random_state=42,        
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features = max_features,
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf
    )
    pipeline = Pipeline(steps=[('sampling', oversample),
                      ('classifier', model)])
    
    pipeline.fit(x_train_tf, y_train)
    preds_val = model.predict(x_val_tf)
    score = f1_score(y_val, preds_val)
    return score

In [14]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=100)

[I 2022-04-13 03:30:12,171] A new study created in memory with name: no-name-1c3f2a30-b302-48cc-a0f0-f5c5cfbe0665
[I 2022-04-13 03:30:15,193] Trial 0 finished with value: 0.95 and parameters: {'max_depth': 10, 'n_estimators': 500, 'max_leaf_nodes': 15, 'min_samples_split': 12}. Best is trial 0 with value: 0.95.
[I 2022-04-13 03:30:15,915] Trial 1 finished with value: 0.9400000000000001 and parameters: {'max_depth': 6, 'n_estimators': 185, 'max_leaf_nodes': 9, 'min_samples_split': 48}. Best is trial 0 with value: 0.95.
[I 2022-04-13 03:30:18,087] Trial 2 finished with value: 0.9400000000000001 and parameters: {'max_depth': 10, 'n_estimators': 281, 'max_leaf_nodes': 23, 'min_samples_split': 16}. Best is trial 0 with value: 0.95.
[I 2022-04-13 03:30:18,587] Trial 3 finished with value: 0.9215686274509804 and parameters: {'max_depth': 9, 'n_estimators': 183, 'max_leaf_nodes': 2, 'min_samples_split': 37}. Best is trial 0 with value: 0.95.
[I 2022-04-13 03:30:19,881] Trial 4 finished with va

In [15]:
'''
Trial 30 finished with value: 0.964467005076142
{'max_depth': 10,
 'max_leaf_nodes': 5,
 'min_samples_split': 4,
 'n_estimators': 316}

 '''

study.best_params

{'max_depth': 10,
 'max_leaf_nodes': 16,
 'min_samples_split': 4,
 'n_estimators': 466}

In [16]:
best_params = study.best_params
model = RandomForestClassifier(
        random_state=42,        
        **best_params
    )
pipeline = Pipeline(steps=[('sampling', oversample),
                    ('classifier', model)])

pipeline.fit(x_train_tf, y_train)
preds_test = model.predict(x_test_tf)
f1score = f1_score(y_test, preds_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, preds_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       225
           1       0.95      0.99      0.97       113

    accuracy                           0.98       338
   macro avg       0.97      0.98      0.98       338
weighted avg       0.98      0.98      0.98       338



In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       225
           1       0.95      0.99      0.97       113

    accuracy                           0.98       338
   macro avg       0.97      0.98      0.98       338
weighted avg       0.98      0.98      0.98       338



# Pipepline 2, handle category value

In [18]:
# cat_label_cols = ['CST_PERF_ST', 'CST_MKT_SEG']
cat_ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('encoder', OrdinalEncoder() 
    )])

# cat_onehot_cols = [f for f in cat_cols if f not in cat_label_cols]
cat_onehot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_val')),
    ('encoder', OneHotEncoder(handle_unknown = 'ignore', sparse=False) 
    )])

preprocessor = ColumnTransformer(
    transformers=[
                  ('num', numeric_transformer, num_cols), 
                  ('cat_label', cat_ordinal_transformer, object_cols)
                  #('cat_onehot', cat_onehot_transformer, ['CLOSE_RSN','CHIP_YN']) # ['CLOSE_RSN', 'COMM_FREQ']
                 ], remainder='drop')

In [24]:
preprocessor.fit(x_train)

x_train_tf = pd.DataFrame(preprocessor.transform(x_train), columns = get_feature_names(preprocessor)) 
x_val_tf = pd.DataFrame(preprocessor.transform(x_val), columns = get_feature_names(preprocessor)) 
x_test_tf = pd.DataFrame(preprocessor.transform(x_test), columns = get_feature_names(preprocessor)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer cat_label (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer cat_label (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages

In [20]:
def run(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)    
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    max_features = trial.suggest_int('max_leaf_nodes', 2, 30)    
    min_samples_split = trial.suggest_int('min_samples_split', 2, 50) 
    min_samples_leaf = trial.suggest_int('min_samples_split', 2, 50)

    model = RandomForestClassifier(
        random_state=42,        
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features = max_features,
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf
    )
    pipeline = Pipeline(steps=[('sampling', oversample),
                      ('classifier', model)])
    
    pipeline.fit(x_train_tf, y_train)
    preds_val = model.predict(x_val_tf)
    score = f1_score(y_val, preds_val)
    return score

In [21]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=100)

[I 2022-04-13 03:33:11,477] A new study created in memory with name: no-name-9a32d5ed-297a-4120-a5f7-57584bdea400
[I 2022-04-13 03:33:12,107] Trial 0 finished with value: 0.9547738693467336 and parameters: {'max_depth': 8, 'n_estimators': 134, 'max_leaf_nodes': 9, 'min_samples_split': 13}. Best is trial 0 with value: 0.9547738693467336.
[I 2022-04-13 03:33:14,840] Trial 1 finished with value: 0.9447236180904524 and parameters: {'max_depth': 7, 'n_estimators': 487, 'max_leaf_nodes': 18, 'min_samples_split': 24}. Best is trial 0 with value: 0.9547738693467336.
[I 2022-04-13 03:33:15,446] Trial 2 finished with value: 0.9447236180904524 and parameters: {'max_depth': 10, 'n_estimators': 143, 'max_leaf_nodes': 9, 'min_samples_split': 44}. Best is trial 0 with value: 0.9547738693467336.
[I 2022-04-13 03:33:15,908] Trial 3 finished with value: 0.9082125603864734 and parameters: {'max_depth': 2, 'n_estimators': 119, 'max_leaf_nodes': 16, 'min_samples_split': 9}. Best is trial 0 with value: 0.95

In [22]:
'''
0.9595959595959597
{'max_depth': 9,
 'max_leaf_nodes': 11,
 'min_samples_split': 10,
 'n_estimators': 219}

'''
study.best_params

{'max_depth': 10,
 'max_leaf_nodes': 29,
 'min_samples_split': 2,
 'n_estimators': 402}

In [25]:
best_params = study.best_params
model = RandomForestClassifier(
        random_state=42,        
        **best_params
    )
pipeline = Pipeline(steps=[('sampling', oversample),
                    ('classifier', model)])

pipeline.fit(x_train_tf, y_train)
preds_test = model.predict(x_test_tf)
f1score = f1_score(y_test, preds_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, preds_test))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       225
           1       0.96      0.99      0.97       113

    accuracy                           0.98       338
   macro avg       0.98      0.98      0.98       338
weighted avg       0.98      0.98      0.98       338

